Assignment 2

The file ‘student_scores.csv’ provides sample data of students and their test score along with attendance and project status. This is available in a CSV file. Use this content and create a python program (using pandas) for the following requirements. Make them as functions in a module.

    1. Function that reads the CSV file and returns subject wise topper(s) and overall topper(s), such that:
        a. Toppers shall have at least 60% attendance
        b. Toppers shall have the project submitted.

    2. A function that returns a data frame with the following columns added along with original data:
        a. ‘Average Score’ : For each student
        b. ‘Grade’ :  based on average score (A : >= 90; B : 75 .. 89.99; C : 60 .. 74.99; D : <60)
        c. ‘Performance’ : 
            ▪ ‘Excellent’ : Grade A and attendance > 90%, project submitted
            ▪ ‘Needs Attention’ : Grade D OR project not submitted OR attendance < 60%
            ▪ ‘Satisfactory’ : All others
    3. A function that exports the summary statistics of the subject wise marks, attendance to a CSV file.

In [ ]:
import pandas as pd
import os

In [ ]:
def get_subj_toppers_overall_toppers(filepath):
   
   df=pd.read_csv(filepath)
   eligibility_criteria= df[(df['Attendance (%)'] >= 60) & (df['Project Submitted'] == True)]
   subjects= df.columns[1:4]
   subject_wise_topper={}
   overall_toppers={}
   for subject in subjects:        
        max_score_subject=eligibility_criteria[subject].max()
        toppers=eligibility_criteria.loc[eligibility_criteria[subject] == max_score_subject, 'Name'].tolist()
        subject_wise_topper[subject]= toppers
    #Overall Toppers
   averages = eligibility_criteria[subjects].mean(axis=1)
   max_avg = averages.max()
   overall_toppers = eligibility_criteria.loc[averages == max_avg, 'Name'].tolist()
   #overall_toppers.append("Overall Toppers:")
   return {
        'Subject wise Topper(s)': subject_wise_topper,
        'Overall Topper(s)': overall_toppers
    }   
    

In [ ]:
def get_addtional_data(filepath):
      
    df=pd.read_csv(filepath)    
    subjects= df.columns[1:4]

    # 1) ‘Average Score’ : For each student
    df['Average Score'] = df[subjects].mean(axis=1)

    # 2) ‘Grade’ :  based on average score (A : >= 90; B : 75 .. 89.99; C : 60 .. 74.99; D : <60)
    def mark_grade(avg):
        if avg >= 90:
            return 'A'
        elif avg >= 75:
            return 'B'
        elif avg >= 60:
            return 'C'
        else:
            return 'D'

    df['Grade'] = df['Average Score'].apply(mark_grade)

    # 3) ‘Performance’ : 
        #    ▪ ‘Excellent’ : Grade A and attendance > 90%, project submitted
        #    ▪ ‘Needs Attention’ : Grade D OR project not submitted OR attendance < 60%
        #    ▪ ‘Satisfactory’ : All others
    def mark_performance(row):
        if row['Grade'] == 'A' and row['Attendance (%)'] > 90 and row['Project Submitted']:
            return 'Excellent'
        elif row['Grade'] == 'D' or not row['Project Submitted'] or row['Attendance (%)'] < 60:
            return 'Needs Attention'
        else:
            return 'Satisfactory'

    df['Performance'] = df.apply(mark_performance, axis=1)

    return df

In [68]:
def exports_summary_statistics(inputfile, output_file):
    """
    Exports summary statistics (count, mean, std, min, 25%, 50%, 75%, max)
    for Math, Science, English, Attendance (%) to a CSV file.
    """
    print(f"Exporting summary statistics to {output_file}")
    df=pd.read_csv(inputfile)
    statistics_cols= df.columns[1:5]    #['Math', 'Science', 'English', 'Attendance (%)']
    #statistics_cols.insert(0,'Stat')
    statistics = df[statistics_cols].describe().round(2)
    statistics = statistics.reset_index().rename(columns={'index': 'Stat'})
    reordered = ['Stat'] + [c for c in statistics.columns if c != 'Stat']
    statistics = statistics[reordered]
    statistics.to_csv(output_file)
    print(f"Summary statistics exported successfully.- {output_file}")
    return statistics

In [70]:
csv_filePath="student_scores.csv"
csv_output_file="assinment2_summary_statistics.csv"
#check for file existance 
isFileExist=os.path.exists(csv_filePath)
if(isFileExist):
    print(f"Given CSV file Path {csv_filePath} exist!")
    print("=====" * 30)    
    result_subj_toppers_overall_toppers= get_subj_toppers_overall_toppers(csv_filePath)
    print("(Q-2.1)")
    print(result_subj_toppers_overall_toppers)
    print("=====" * 30)    
    result_addtional_data = get_addtional_data(csv_filePath)    
    print("(Q-2.2)")
    print(result_addtional_data)
    print("=====" * 30)   
    result_Statistics = exports_summary_statistics(csv_filePath, csv_output_file)
    print("(Q-2.3)")
    print(result_Statistics)
    print("=====" * 30) 
else:
     print(f"Given CSV file Path {csv_filePath} does not exist, Please check path.")

Given CSV file Path student_scores.csv exist!
(Q-2.1)
{'Subject wise Topper(s)': {'Math': ['Rachel Mcneil', 'Ashley Garcia', 'Heidi Edwards'], 'Science': ['Benjamin Stein'], 'English': ['Warren Harris']}, 'Overall Topper(s)': ['Ashley Garcia']}
(Q-2.2)
                Name  Math  Science  English  Attendance (%)  \
0       Robert Roman    78       93       96           96.55   
1     Joseph Sanchez    91       62       60           98.71   
2     Christina Hall    68       76       57           99.80   
3          Ann Brown    54       79       94           52.79   
4     Thomas Herrera    82       96       86           86.85   
..               ...   ...      ...      ...             ...   
195     Brian Jacobs    82       39       97           93.42   
196    Jacob Walters    68       40       67           54.23   
197       Eric Curry    75       81       60           79.86   
198  Sara Valdez DDS    52       89       83           99.31   
199   Jessica Barnes    71       74       8